# Transfer learning with a pretrained ConvNet

사전 훈련 된 네트워크(MobileNet V2 model)에서 전이 학습을 사용하여 고양이와 강아지의 이미지를 분류하는 방법을 학습

사전 훈련 된 모델은 이전에 대규모 데이터 세트, 일반적으로 대규모 이미지 분류 작업에서 학습되어 저장된 네트워크

(1) 특징 추출을 위해 사전 훈련 된 모델 사용 : 작은 데이터 세트로 작업 할 때 동일한 도메인에서 더 큰 데이터 세트에 대해 훈련 된 모델에서 학습 한 기능을 활용하는 것이 일반적입니다. 사전 훈련 된 모델을 인스턴스화하고 완전히 연결된 분류기를 맨 위에 추가하면됩니다. 사전 훈련 된 모델은 "동결"되며 분류기의 가중치 만 훈련 중에 업데이트됩니다. 이 경우 컨벌루션베이스는 각 이미지와 관련된 모든 기능을 추출했으며 추출 된 기능 세트가 제공된 이미지 클래스를 결정하는 분류기를 훈련합니다.

(2)사전 훈련 된 모델 미세 조정 : 성능을 더욱 향상시키기 위해 사전 훈련 된 모델의 최상위 계층을 미세 조정을 통해 새로운 데이터 세트로 재사용 할 수 있습니다. 이 경우 모델이 데이터 세트와 관련된 고급 기능을 학습 할 수 있도록 가중치를 조정했습니다. 이 기술은 일반적으로 훈련 데이터 세트가 크고 사전 훈련 된 모델이 훈련 된 원래 데이터 세트와 매우 유사한 경우에 권장됩니다.

In [ ]:
import os

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf


## Data preprocessing

### Data download

Use [TensorFlow Datasets](http://tensorflow.org/datasets) to load the cats and dogs dataset.

`tfds` package :  load pre-defined data

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

fds.load()로 데이터를 다운로드하여 'cats_vs_dogs'로 부터 읽어서  tf.data.Dataset오브젝트를 리턴   

(train, validation, test) : 학습,검증,테스트 데이터를 80 %, 10 %,10 % 로 분할

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

`tf.data.Dataset`은 `(image, label)` 을 포함, 3 channels

In [ ]:
print(raw_train)
print(raw_validation)
print(raw_test)

훈련 세트에서 처음 두 개의 이미지와 레이블을 보임

In [ ]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

### Format the Data
이미지를 고정 된 입력 크기로 조정하고 입력 채널의 범위를 [-1,1]로 처리

In [ ]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

map 메소드를 사용하여 데이터 세트의 각 항목에이 함수를 적용

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Now shuffle and batch the data.

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

데이터 확인

In [ ]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

## Create the base model from the pre-trained convnets(MobileNet V2)
사전 훈련 된 MobileNet V2로 기본 모델 생성  

Google에서 개발 한 MobileNet V2 모델 : 1.4M 이미지와 1000 개의 클래스로 구성된 대규모 데이터 세트 인 ImageNet 데이터 세트에 대해 사전 학습된 모델  
- MobileNet V2의 특징추출 계층(하위 계층)만 이용  

- ImageNet에 대해 가중치가 사전로드 된 MobileNet V2 모델 생성시   include_top = False 로 지정하여 특징 계층만 사용  

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

 160x160x3이미지를 5x5x1280 특징 블록 으로 변환

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Feature extraction
convolutional base를 사전 학습된  모델로 고정하고 top-level classifier만 학습

### Freeze the convolutional base
모델을 컴파일하고 훈련시키기 전에 convolutional base를 고정(layer.trainable = False)  
훈련 중 지정된 레이어의 가중치가 수정안됨

In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

GlobalAveragePooling2D layer에서  5x5공간 위치에 대한 평균을 생성하고 이미지당 1280개의 노드 생성

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

tf.keras.layers.Dense layer를 이용하여 하나의 출력을 설정

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

tf.keras.Sequential model에  
-feature extractor(base_model)  
-GlobalAveragePooling2D layer  
-prediction_layer를 포함

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

### Compile the model

RMSprop, binary cross-entropy을 이용하여 모델 컴파일

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Train the model


In [ ]:
initial_epochs = 10
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

### Learning curves


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tuning (미세조정)

사전 훈련 된 모델의 최상위 레이어의 가중치를 훈련 (또는 "미세 조정")
학습을 통해 가중치는 일반 특징 맵에서 추가된 데이터 집합과 관련된 특징으로 강제 조정

### Un-freeze the top layers of the model


base_model 을 학습가능하도록 설정

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Compile the model

학습율을 낮추어 모델 컴파일

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continue training the model

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()